In [1]:
import pandas as pd
import numpy as np
import os
import collections
from tqdm import tqdm 

In [ ]:
#TODO clear code

In [2]:
save = "twitter_features"
folder = "twitter/twitter"
apath="snap_datasets/"

In [3]:
def process_file(file_id, folder):
    # Read ego, egofeat, and featnames data from the file
    ego = pd.read_csv(apath+f"{folder}/{file_id}.egofeat", sep=" ", header=None)
    egofeat = pd.read_csv(apath+f"{folder}/{file_id}.feat", sep=" ", header=None)
    featnames = pd.read_csv(apath+f"{folder}/{file_id}.featnames", sep=" ", header=None)
 
    # Create a list of ego features
    eglist=[file_id] #set 1th element as fileid
    for i in list(ego):
        eglist.append(int(ego[i].iloc[0])) #features list from ego
    egofeat.loc[len(egofeat.index)] = eglist #combine .egofeat with .feat
    
    #In case of the bad featurename split from csv edit the column
    featnames['feat_name'] = featnames.apply(lambda row: f"{row[1]}", axis=1)
    featnames = featnames.drop([1], axis=1)
    featnames[0] += 1 #reallign columns
    featnames.loc[len(featnames.index)] = [0, 'id']#set 0 as id
    featnames = featnames.sort_values(by=[0])#sort values
    
    a = list(featnames['feat_name'])
    egofeat.columns = a #change the column names to match feature name 
    
    df_boolean = egofeat.iloc[:, 1:].astype(bool)

# Concatenate the first column with the boolean DataFrame
    res = pd.concat([egofeat.iloc[:, :1], df_boolean], axis=1)

    return res

file_ids = []
for filename in os.listdir(apath+f"{folder}"):
    if filename.endswith(".egofeat"):
        file_id = int(filename.split(".")[0])
        file_ids.append(file_id)  
        
egofeat_list = [process_file(file_id, folder) for file_id in file_ids]

In [ ]:
# egofeat_list

In [ ]:
# def process_feature(file_id, folder):
#     featnames = pd.read_csv(apath+f"{folder}/{file_id}.featnames", sep=" ", header=None)
#     return list(featnames[1])
# file_ids = []
# features_column = [process_feature(file_id, folder) for file_id in file_ids]
# flattened_list = [item for sublist in egofeat_list for item in sublist]
# # Count the occurrences of each item
# a_counter = collections.Counter(flattened_list)
# # Sort items by their count
# sorted_by_keys = sorted(a_counter.items(), key=lambda x: x[1])
# # Remove items with count 1
# list_of_remove = list(filter(lambda x: x[1] <= 1, sorted_by_keys))
# # Extract the keys (items) from the tuples
# a = [item[0] for item in list_of_remove]

In [ ]:
# a

In [ ]:
# for i in range(len(egofeat_list)):
#     egofeat_list[i] = egofeat_list[i].drop(columns=a, errors='ignore')

In [4]:
len(egofeat_list)

973

In [5]:
data=[]
with tqdm(total=len(egofeat_list), position=0, leave=True) as pbar:
    for x in egofeat_list:
        ego={}
        for eid in list(x['id']):
            row = x[x['id'] == eid]
            feats=list(row.replace(False,np.nan).T.dropna().T)
            feats.remove('id')
            ego[str(eid)]=feats
        data.append(ego)
        pbar.update()
        
        
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 973/973 [2:19:23<00:00,  8.60s/it]


In [6]:
from collections import defaultdict

# Group dictionaries by keys and merge their lists
merged_dicts = defaultdict(list)
with tqdm(total=len(data), position=0, leave=True) as pbar:
    for dictionary in data:
        for key, value in dictionary.items():
            first_list=merged_dicts[key]
            merged_dicts[key]=first_list + list(set(value) - set(first_list))
        pbar.update()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 973/973 [00:00<00:00, 1405.01it/s]


In [24]:
flattened={}
for x in merged_dicts:
    val= [f for f in merged_dicts[x]]
    stringval = " ".join(val)
    flattened[x]=stringval

In [9]:
keys_list = list(flattened.keys())
values_list = list(flattened.values())
gg={"id":keys_list,"features":values_list}

In [10]:
gg

{'id': ['21232196',
  '45002767',
  '22636391',
  '16652840',
  '42986082',
  '23154457',
  '3298931',
  '14261656',
  '19781217',
  '26527170',
  '9451052',
  '75662159',
  '75362374',
  '15797829',
  '18382430',
  '19069709',
  '40244876',
  '17354254',
  '42840044',
  '28716937',
  '69406740',
  '69846835',
  '15902958',
  '15900177',
  '16391766',
  '15197542',
  '16505525',
  '78417564',
  '23240788',
  '61196466',
  '35216869',
  '81704859',
  '7895822',
  '18630926',
  '96247161',
  '7861312',
  '15462407',
  '15191880',
  '18319945',
  '24911468',
  '104804951',
  '102400865',
  '14159157',
  '78415551',
  '18988524',
  '7953002',
  '82164174',
  '18941809',
  '25191293',
  '17052055',
  '44308344',
  '14584466',
  '24615066',
  '20934243',
  '79680347',
  '27614577',
  '90820354',
  '32934942',
  '19792993',
  '6875402',
  '62042473',
  '21106726',
  '28145787',
  '78088588',
  '76236732',
  '61240263',
  '19643522',
  '7834942',
  '18927441',
  '16364081',
  '20680992',
  '19

In [16]:

f=pd.DataFrame(data=gg,  index=gg['id'])
f.to_csv("twitter_features.csv",index=None)

In [15]:
f

,id,features
21232196,21232196,#sega #iphone #Nintendo #games #wii #Capcom #i...
45002767,45002767,#Nintendo #Pokemon
22636391,22636391,
16652840,16652840,@bradshoemaker @SavageHands @cotgw @retrowaret...
42986082,42986082,#Wii
...,...,...
50408942,50408942,
21506325,21506325,
20092984,20092984,@urbanabusiness: @cg_food_coop: @CentralILFood...
50545785,50545785,@charlesweinberg: @cdcurtiss #oscars @SmilePol...
